In [1]:
!pip install -q sentence-transformers pylatexenc octis

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.
tensorflow 2.6.3 requires absl-py~=0.10, but you have absl-py 1.0.0 which is incompatible.
tensorflow 2.6.3 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow 2.6.3 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow 2.6.3 requires wrapt~=1.12.1, but you have wrapt 1.14.0 which is incompatible.
tensorflow-transform 1.7.0 requires pyarrow<6,>=1, but you have pyarrow 7.0.0 which is incompatible.
tensorflow-transform 1.7.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,<2.9,>=1.15.5, but you have tensorflow 2.6.3 which is incompati

In [2]:
!pip install -q hdbscan

In [3]:
from transformers import AutoTokenizer, AutoModel
import umap
from sklearn.cluster import DBSCAN, OPTICS
from sklearn.metrics import silhouette_score, davies_bouldin_score
import optuna
import hdbscan
from numpy.random import default_rng
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import torch
import optuna

from pylatexenc.latex2text import LatexNodes2Text

In [4]:
tqdm.pandas()
docs_df = pd.read_csv('../input/task-2-data/to_use.csv')
df = docs_df.sample(15000, random_state=42)
rng = default_rng(49)
df['type_set'] = rng.choice(np.array(['train', 'val', 'test']), 15000)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
def clean_abstract(txt, stp_wrds):
    txt = txt.lower()
    try:
        txt = LatexNodes2Text().latex_to_text(txt)
    except:
        txt = txt.lower()
    sup = word_tokenize(txt, language="english")
    lemmatizer = WordNetLemmatizer()
    sup = [lemmatizer.lemmatize(word) for word in sup]
    ret = []
    for el in sup:
        el.replace('\\', '')
        if el not in stp_wrds and el not in string.punctuation:
            ret.append(el)
    return ' '.join(ret)


stop_words = nltk.corpus.stopwords.words("english")
stop_words += ['we', 'paper', 'new', 'article', "''", "``", "”", 'et', 'al', 'study', 'state', 'of', 'the', 'art']
cleaned_abstract = df['abstract'].progress_apply(clean_abstract, args=[stop_words])
df['abstract'] = cleaned_abstract

  0%|          | 0/15000 [00:00<?, ?it/s]

In [6]:
train = df[df['type_set'] == 'train']['abstract'].tolist()
val = df[df['type_set'] == 'val']['abstract'].tolist()
test = df[df['type_set'] == 'test']['abstract'].tolist()

In [7]:
untrained_models = ['johngiorgi/declutr-base', 'google/electra-base-discriminator', 'microsoft/deberta-v3-small', 'xlnet-base-cased']
trained_models = ['../input/fine-tune-mlm/MLM_mmodel_w/' + x for x in untrained_models[:-1]]
trained_models

['../input/fine-tune-mlm/MLM_mmodel_w/johngiorgi/declutr-base',
 '../input/fine-tune-mlm/MLM_mmodel_w/google/electra-base-discriminator',
 '../input/fine-tune-mlm/MLM_mmodel_w/microsoft/deberta-v3-small']

In [8]:
@torch.no_grad()
def extract_emb(txt, tok, emb):
    inputs_ = tok(txt, return_tensors="pt", truncation=True).to('cuda')
    outputs_ = emb(**inputs_)
    ls = outputs_.last_hidden_state
    cls =  ls[:, 0, :].clone().detach()
    return cls.cpu().numpy()[0]

untrained_dict = {}
for md in untrained_models:
    tokenizer = AutoTokenizer.from_pretrained(md)
    model = AutoModel.from_pretrained(md).cuda()

    emb_val = [extract_emb(el, tokenizer, model) for el in tqdm(val, desc='Processing val')]
    emb_test = [extract_emb(el, tokenizer, model) for el in tqdm(test, desc='Processing test')]

    untrained_dict[md] = (emb_val, emb_test)


trained_dict = {}
for i, md in enumerate(trained_models):
    tokenizer = AutoTokenizer.from_pretrained(untrained_models[i])
    model = AutoModel.from_pretrained(md).cuda()

    emb_val = [extract_emb(el, tokenizer, model) for el in tqdm(val, desc='Processing val')]
    emb_test = [extract_emb(el, tokenizer, model) for el in tqdm(test, desc='Processing test')]

    trained_dict[md] = (emb_val, emb_test)

Downloading:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

Processing test:   0%|          | 0/4992 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at ../input/fine-tune-mlm/MLM_mmodel_w/microsoft/deberta-v3-small were not used when initializing DebertaV2Model: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Processing val:   0%|          | 0/4967 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Processing test:   0%|          | 0/4992 [00:00<?, ?it/s]

# Pipeline for clustering

In [9]:
def objective_dbscan(trial):
    min_dist = trial.suggest_float("min_dist", 0.01, 1, log=True)
    n_neigh = trial.suggest_int("n_neighbors", 2, 200, log=True)
    n_comp = trial.suggest_int("n_components", 10, 250, log=True)
    clusterable_embedding = umap.UMAP(
        n_neighbors=n_neigh,
        min_dist=min_dist,
        n_components=n_comp,
        random_state=42,
    ).fit_transform(x_val)
    eps = trial.suggest_float("eps", 0.001, 1, log=True)
    ms = trial.suggest_int("min_samples", 2, 100, log=True)
    cluster_obj = DBSCAN(eps=eps, min_samples=ms)

    labels = cluster_obj.fit(clusterable_embedding).labels_
    if np.unique(labels).shape[0] < 10:
        return 0
    try:
        score = silhouette_score(clusterable_embedding, labels)
        return score
    except ValueError:
        return 0


def objective_hdbscan(trial):
    min_dist = trial.suggest_float("min_dist", 0.01, 1, log=True)
    n_neigh = trial.suggest_int("n_neighbors", 2, 200, log=True)
    n_comp = trial.suggest_int("n_components", 10, 250, log=True)
    clusterable_embedding = umap.UMAP(
        n_neighbors=n_neigh,
        min_dist=min_dist,
        n_components=n_comp,
        random_state=42,
    ).fit_transform(x_val)
    cse = trial.suggest_float("cluster_selection_epsilon", 0.001, 1, log=True)
    mcs = trial.suggest_int("min_cluster_size", 2, 100, log=True)
    ms = trial.suggest_int("min_samples", 2, 40, log=True)
    cluster_obj = hdbscan.HDBSCAN(min_samples=ms, min_cluster_size=mcs, cluster_selection_epsilon=cse)

    labels = cluster_obj.fit(clusterable_embedding).labels_
    if np.unique(labels).shape[0] < 10:
        return 0
    try:
        score = silhouette_score(clusterable_embedding, labels)
        return score
    except ValueError:
        return 0


def objective_optics(trial):
    min_dist = trial.suggest_float("min_dist", 0.01, 1, log=True)
    n_neigh = trial.suggest_int("n_neighbors", 2, 200, log=True)
    n_comp = trial.suggest_int("n_components", 10, 250, log=True)
    clusterable_embedding = umap.UMAP(
        n_neighbors=n_neigh,
        min_dist=min_dist,
        n_components=n_comp,
        random_state=42,
    ).fit_transform(x_val)
    max_eps = trial.suggest_float("max_eps", 0.001, 1, log=True)
    ms = trial.suggest_int("min_samples", 2, 100, log=True)
    cluster_obj = OPTICS(max_eps=max_eps, min_samples=ms)

    labels = cluster_obj.fit(clusterable_embedding).labels_
    if np.unique(labels).shape[0] < 10:
        return 0
    try:
        score = silhouette_score(clusterable_embedding, labels)
        return score
    except ValueError:
        return 0

## Untrained

In [10]:
x_val = None

untrained_params = []
for el in tqdm(untrained_models):
    tmp = {}
    x_val = untrained_dict[el][0]

    # dbscan
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_dbscan, n_trials=25, n_jobs=8, show_progress_bar=True)
    tmp['dbscan'] = study.best_trial.params

    # hdbscan
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_hdbscan, n_trials=25, n_jobs=8, show_progress_bar=True)
    tmp['hdbscan'] = study.best_trial.params

    # optics
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_optics, n_trials=25, n_jobs=8, show_progress_bar=True)
    tmp['optics'] = study.best_trial.params

    untrained_params.append(tmp)

  0%|          | 0/4 [00:00<?, ?it/s]

[I 2022-05-10 18:01:05,261] A new study created in memory with name: no-name-8e55bac3-10d3-4b9d-8e3f-c90f1db1fa2e
/opt/conda/lib/python3.7/site-packages/optuna/study/study.py:397: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/optuna/study/_optimize.py:80: UserWarning: Progress bar only supports serial execution (`n_jobs=1`).
  warnings.warn("Progress bar only supports serial execution (`n_jobs=1`).")
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[I 2022-05-10 18:03:23,416] Trial 7 finished with value: 0.0 and parameters: {'min_dist': 0.7249800429944587, 'n_neighbors': 7, 'n_components': 93, 'eps': 0.5956207231933537, 'min_samples': 22}. Best is trial 7 with value: 0.0.
[I 2022-05-10 18:03:27,479] Trial 6 finished with value: 0.0 and parameters: {'min_dist': 0.7331433656507167, 'n_neighbors': 7, 'n_components': 75, 'eps': 0.00860654032751899, 'min_samples': 13}. Best is trial 7 with value: 0.0.
[I 2022-05-10 18:03:37,178] Trial 2 finished with value: 0.0 and parameters: {'min_dist': 0.03469762079994428, 'n_neighbors': 16, 'n_components': 24, 'eps': 0.40348182439820307, 'min_samples': 20}. Best is trial 7 with value: 0.0.
[I 2022-05-10 18:06:01,576] Trial 4 finished with value: 0.0 and parameters: {'min_dist': 0.036556873147991, 'n_neighbors': 31, 'n_components': 196, 'eps': 0.0041019013680141615, 'min_samples': 19}. Best is trial 7 with value: 0.0.
[I 2022-05-10 18:06:11,570] Trial 1 finished with value: 0.0 and parameters: {'m

In [11]:
untrained_params

[{'dbscan': {'min_dist': 0.05695967860851086,
   'n_neighbors': 2,
   'n_components': 108,
   'eps': 0.037646398138614326,
   'min_samples': 2},
  'hdbscan': {'min_dist': 0.05136406857737105,
   'n_neighbors': 2,
   'n_components': 33,
   'cluster_selection_epsilon': 0.039140304353105185,
   'min_cluster_size': 17,
   'min_samples': 24},
  'optics': {'min_dist': 0.054311144164378676,
   'n_neighbors': 3,
   'n_components': 10,
   'max_eps': 0.018754624619633533,
   'min_samples': 6}},
 {'dbscan': {'min_dist': 0.12128207760846393,
   'n_neighbors': 2,
   'n_components': 105,
   'eps': 0.30067938715390713,
   'min_samples': 9},
  'hdbscan': {'min_dist': 0.1533167482873666,
   'n_neighbors': 2,
   'n_components': 57,
   'cluster_selection_epsilon': 0.04500969081889752,
   'min_cluster_size': 15,
   'min_samples': 14},
  'optics': {'min_dist': 0.722315803786614,
   'n_neighbors': 7,
   'n_components': 35,
   'max_eps': 0.1151122964088759,
   'min_samples': 3}},
 {'dbscan': {'min_dist': 0.0

## Trained

In [12]:
x_val = None

trained_params = []
for el in tqdm(trained_models):
    tmp = {}
    x_val = trained_dict[el][0]

    # dbscan
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_dbscan, n_trials=25, n_jobs=8, show_progress_bar=True)
    tmp['dbscan'] = study.best_trial.params

    # hdbscan
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_hdbscan, n_trials=25, n_jobs=8, show_progress_bar=True)
    tmp['hdbscan'] = study.best_trial.params

    # optics
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_optics, n_trials=25, n_jobs=8, show_progress_bar=True)
    tmp['optics'] = study.best_trial.params

    trained_params.append(tmp)

  0%|          | 0/3 [00:00<?, ?it/s]

[I 2022-05-10 22:01:27,661] A new study created in memory with name: no-name-b7abb403-261e-407b-b4a2-c51fe3f1cc5c
[I 2022-05-10 22:02:12,680] Trial 6 finished with value: 0.0 and parameters: {'min_dist': 0.10906803588467563, 'n_neighbors': 3, 'n_components': 34, 'eps': 0.021430130766911377, 'min_samples': 9}. Best is trial 6 with value: 0.0.
[I 2022-05-10 22:02:20,765] Trial 3 finished with value: -0.23013997077941895 and parameters: {'min_dist': 0.03957517298813015, 'n_neighbors': 3, 'n_components': 46, 'eps': 0.6417943497548193, 'min_samples': 5}. Best is trial 6 with value: 0.0.
[I 2022-05-10 22:02:21,385] Trial 2 finished with value: 0.0 and parameters: {'min_dist': 0.10508573344677774, 'n_neighbors': 3, 'n_components': 40, 'eps': 0.022029525444773344, 'min_samples': 60}. Best is trial 6 with value: 0.0.
[I 2022-05-10 22:02:59,372] Trial 7 finished with value: 0.0 and parameters: {'min_dist': 0.017997511734356895, 'n_neighbors': 3, 'n_components': 195, 'eps': 0.003173778751996743, 

In [13]:
trained_params

[{'dbscan': {'min_dist': 0.10906803588467563,
   'n_neighbors': 3,
   'n_components': 34,
   'eps': 0.021430130766911377,
   'min_samples': 9},
  'hdbscan': {'min_dist': 0.031167800955051728,
   'n_neighbors': 2,
   'n_components': 127,
   'cluster_selection_epsilon': 0.04073616659880058,
   'min_cluster_size': 4,
   'min_samples': 3},
  'optics': {'min_dist': 0.029884042357306583,
   'n_neighbors': 2,
   'n_components': 116,
   'max_eps': 0.2956953731275992,
   'min_samples': 7}},
 {'dbscan': {'min_dist': 0.05154877854616341,
   'n_neighbors': 2,
   'n_components': 41,
   'eps': 0.1782554733595419,
   'min_samples': 11},
  'hdbscan': {'min_dist': 0.010256277101590152,
   'n_neighbors': 38,
   'n_components': 13,
   'cluster_selection_epsilon': 0.09241753201045283,
   'min_cluster_size': 2,
   'min_samples': 34},
  'optics': {'min_dist': 0.9471429310427595,
   'n_neighbors': 3,
   'n_components': 62,
   'max_eps': 0.021999048476075037,
   'min_samples': 35}},
 {'dbscan': {'min_dist': 0

In [14]:
def extract_params(params):
    umap_params = {'min_dist': params['min_dist'],
                   'n_neighbors': params['n_neighbors'],
                   'n_components': params['n_components'],
                   }
    exclude_keys = {'min_dist', 'n_neighbors', 'n_components'}
    clust_params = {k: params[k] for k in set(list(params.keys())) - exclude_keys}
    return umap_params, clust_params

In [15]:
extract_params(trained_params[0]['dbscan'])

({'min_dist': 0.10906803588467563, 'n_neighbors': 3, 'n_components': 34},
 {'min_samples': 9, 'eps': 0.021430130766911377})

# Results

In [16]:
sup = {'dbscan' : DBSCAN, 'hdbscan' : hdbscan.HDBSCAN, 'optics' : OPTICS}
results_untrained = []
for i, dct in tqdm(enumerate(untrained_params), desc='Processing models'):
    tmp = dict()
    md_name = untrained_models[i]
    x_test = untrained_dict[md_name][1]
    for key in tqdm(list(untrained_params[i].keys()), desc='Processing clusters'):
        umap_pars, clust_pars = extract_params(untrained_params[i][key])
        clusterable_embedding = umap.UMAP(**umap_pars).fit_transform(x_test)
        cluster_obj = sup[key](**clust_pars)
        labels = cluster_obj.fit(clusterable_embedding).labels_
        try:
            score_1 = silhouette_score(clusterable_embedding, labels)
            score_2 = davies_bouldin_score(clusterable_embedding, labels)
        except:
            score_1 = None
            score_2 = None
        tmp[key] = (score_1, score_2)
    results_untrained.append(tmp)
results_untrained

Processing models: 0it [00:00, ?it/s]

Processing clusters:   0%|          | 0/3 [00:00<?, ?it/s]

Processing clusters:   0%|          | 0/3 [00:00<?, ?it/s]

Processing clusters:   0%|          | 0/3 [00:00<?, ?it/s]

Processing clusters:   0%|          | 0/3 [00:00<?, ?it/s]

[{'dbscan': (0.64602375, 0.9866583822807413),
  'hdbscan': (0.23321979, 1.358775597476665),
  'optics': (0.05273408, 0.7928217963777403)},
 {'dbscan': (0.47670454, 0.9807816472033318),
  'hdbscan': (0.39195016, 1.2404838789553978),
  'optics': (None, None)},
 {'dbscan': (None, None),
  'hdbscan': (0.6861681, 1.1864802770477512),
  'optics': (None, None)},
 {'dbscan': (0.89385104, 1.6803089223990693),
  'hdbscan': (0.2935254, 1.1405455062811678),
  'optics': (0.18732232, 1.226008839867279)}]

In [17]:
sup = {'dbscan' : DBSCAN, 'hdbscan' : hdbscan.HDBSCAN, 'optics' : OPTICS}
results_trained = []
for i, dct in tqdm(enumerate(trained_params), desc='Processing models'):
    tmp = dict()
    md_name = trained_models[i]
    x_test = trained_dict[md_name][1]
    for key in tqdm(list(trained_params[i].keys()), desc='Processing clusters'):
        umap_pars, clust_pars = extract_params(trained_params[i][key])
        clusterable_embedding = umap.UMAP(**umap_pars).fit_transform(x_test)
        cluster_obj = sup[key](**clust_pars)
        labels = cluster_obj.fit(clusterable_embedding).labels_
        try:
            score_1 = silhouette_score(clusterable_embedding, labels)
            score_2 = davies_bouldin_score(clusterable_embedding, labels)
        except:
            score_1 = None
            score_2 = None
        tmp[key] = (score_1, score_2)
    results_trained.append(tmp)
results_trained

Processing models: 0it [00:00, ?it/s]

Processing clusters:   0%|          | 0/3 [00:00<?, ?it/s]

Processing clusters:   0%|          | 0/3 [00:00<?, ?it/s]

Processing clusters:   0%|          | 0/3 [00:00<?, ?it/s]

[{'dbscan': (None, None),
  'hdbscan': (0.6695082, 1.104614031975536),
  'optics': (0.46489334, 1.0123356739045715)},
 {'dbscan': (0.034267273, 1.0200794730018719),
  'hdbscan': (-0.25167644, 7.965062549585277),
  'optics': (None, None)},
 {'dbscan': (0.80841964, 0.9295705732871172),
  'hdbscan': (0.6104461, 1.1683270204042073),
  'optics': (0.18511756, 1.1266189269741689)}]

In [18]:
sup = {'dbscan' : DBSCAN, 'hdbscan' : hdbscan.HDBSCAN, 'optics' : OPTICS}
results_trained = []
for i, dct in tqdm(enumerate(trained_params), desc='Processing models'):
    tmp = dict()
    md_name = trained_models[i]
    x_test = trained_dict[md_name][0]
    for key in tqdm(list(trained_params[i].keys()), desc='Processing clusters'):
        umap_pars, clust_pars = extract_params(trained_params[i][key])
        clusterable_embedding = umap.UMAP(**umap_pars).fit_transform(x_test)
        cluster_obj = sup[key](**clust_pars)
        labels = cluster_obj.fit(clusterable_embedding).labels_
        try:
            score_1 = silhouette_score(clusterable_embedding, labels)
            score_2 = davies_bouldin_score(clusterable_embedding, labels)
        except:
            score_1 = None
            score_2 = None
        tmp[key] = (score_1, score_2)
    results_trained.append(tmp)
results_trained

Processing models: 0it [00:00, ?it/s]

Processing clusters:   0%|          | 0/3 [00:00<?, ?it/s]

Processing clusters:   0%|          | 0/3 [00:00<?, ?it/s]

Processing clusters:   0%|          | 0/3 [00:00<?, ?it/s]

[{'dbscan': (None, None),
  'hdbscan': (0.667495, 1.1679090559152299),
  'optics': (0.44966054, 1.0496349907352889)},
 {'dbscan': (0.076374345, 0.9534871413855125),
  'hdbscan': (0.25357938, 1.7824488983923645),
  'optics': (None, None)},
 {'dbscan': (0.8113755, 0.9117667325852178),
  'hdbscan': (0.59281605, 1.1636173076468208),
  'optics': (0.2021489, 1.0778664472361557)}]

In [19]:
# dsds

In [20]:
# df_val = df[df['type_set'] == 'val'].copy()
# X_val = [extract_emb(el, tokenizer, model) for el in tqdm(df_val['abstract'].tolist())]
# clusterable_embedding = umap.UMAP(
#     n_neighbors=2,
#     min_dist=0.4820125085643817,
#     n_components=157,
#     random_state=42,
# ).fit_transform(X_val)
# cluster_obj = DBSCAN(eps=0.9892806110361401, min_samples=5)
# labels = cluster_obj.fit(clusterable_embedding).labels_
# silhouette_score(clusterable_embedding, labels), davies_bouldin_score(clusterable_embedding, labels)

In [21]:
# import matplotlib.pyplot as plt
# import seaborn as sns


# _, ax = plt.subplots(figsize=(30, 5))
# sns.countplot(labels, ax=ax)

In [22]:
# def objective(trial):
#     cluster_name = trial.suggest_categorical("cluster", ["dbscan", "optics", "hdbscan"])
#     min_dist = trial.suggest_float("min_dist", 0.01, 1, log=True)
#     n_neigh = trial.suggest_int("n_neighbors", 2, 200, log=True)
#     n_comp = trial.suggest_int("n_components", 10, 250, log=True)
#     clusterable_embedding = umap.UMAP(
#         n_neighbors=n_neigh,
#         min_dist=min_dist,
#         n_components=n_comp,
#         random_state=42,
#     ).fit_transform(X_train)
#     if cluster_name == "dbscan":
#         eps = trial.suggest_float("eps", 0.001, 1, log=True)
#         ms = trial.suggest_int("min_samples", 2, 100, log=True)
#         cluster_obj = DBSCAN(eps=eps, min_samples=ms)
#     elif cluster_name == "hdbscan":
#         cse = trial.suggest_float("cluster_selection_epsilon", 0.001, 1, log=True)
#         mcs = trial.suggest_int("min_cluster_size", 2, 100, log=True)
#         ms = trial.suggest_int("min_samples", 2, 40, log=True)
#         cluster_obj = hdbscan.HDBSCAN(min_samples=ms, min_cluster_size=mcs, cluster_selection_epsilon=cse)
#     else:
#         max_eps = trial.suggest_float("max_eps", 0.001, 1, log=True)
#         ms = trial.suggest_int("min_samples", 2, 100, log=True)
#         cluster_obj = OPTICS(max_eps=max_eps, min_samples=ms)
#
#     labels = cluster_obj.fit(clusterable_embedding).labels_
#     if np.unique(labels).shape[0] < 10:
#         return 0
#     try:
#         score = silhouette_score(clusterable_embedding, labels)
#         return score
#     except ValueError:
#         return 0
#
